In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torchsummary import summary

In [2]:
# Define transformations for training and validation data
train_transforms = transforms.Compose([
    transforms.Resize((480, 270)),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((480, 270)),
    transforms.ToTensor(),
])

In [3]:
# Load training and validation data
train_data = datasets.ImageFolder('data/train', transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)

test_data = datasets.ImageFolder('data/test', transform=test_transforms)
val_loader = torch.utils.data.DataLoader(test_data, batch_size=256, shuffle=False)

#Load pre-trained ResNet-50
model = models.resnet50(weights='ResNet50_Weights.DEFAULT')

# BackProp prevention :)
for param in model.parameters():
    param.requires_grad = False

# Modify the last layer for our task
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 outputs - base on folder name alphabetical: brake, forward, left right

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# Moving the model to Gpu 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# print(model)
summary(model, input_size=(3 ,480, 270))#Show the model(Resnet50) Structure details 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 240, 135]           9,408
       BatchNorm2d-2         [-1, 64, 240, 135]             128
              ReLU-3         [-1, 64, 240, 135]               0
         MaxPool2d-4          [-1, 64, 120, 68]               0
            Conv2d-5          [-1, 64, 120, 68]           4,096
       BatchNorm2d-6          [-1, 64, 120, 68]             128
              ReLU-7          [-1, 64, 120, 68]               0
            Conv2d-8          [-1, 64, 120, 68]          36,864
       BatchNorm2d-9          [-1, 64, 120, 68]             128
             ReLU-10          [-1, 64, 120, 68]               0
           Conv2d-11         [-1, 256, 120, 68]          16,384
      BatchNorm2d-12         [-1, 256, 120, 68]             512
           Conv2d-13         [-1, 256, 120, 68]          16,384
      BatchNorm2d-14         [-1, 256, 

In [4]:

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# Training loop
epochs = 50  # Iterations

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward() #backprop
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = (correct / total) * 100

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = (correct / total) * 100

    print(f"Epoch {epoch + 1}/{epochs}, "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

 

Epoch 1/50, Train Loss: 1.3139, Train Acc: 38.40%, Val Loss: 1.2767, Val Acc: 51.22%
Epoch 2/50, Train Loss: 1.1430, Train Acc: 60.60%, Val Loss: 1.1669, Val Acc: 49.66%
Epoch 3/50, Train Loss: 1.0162, Train Acc: 67.35%, Val Loss: 1.0565, Val Acc: 60.28%
Epoch 4/50, Train Loss: 0.9187, Train Acc: 72.22%, Val Loss: 0.9616, Val Acc: 65.67%
Epoch 5/50, Train Loss: 0.8427, Train Acc: 74.73%, Val Loss: 0.8660, Val Acc: 71.78%
Epoch 6/50, Train Loss: 0.7816, Train Acc: 76.01%, Val Loss: 0.7881, Val Acc: 74.57%
Epoch 7/50, Train Loss: 0.7322, Train Acc: 78.28%, Val Loss: 0.7206, Val Acc: 78.36%
Epoch 8/50, Train Loss: 0.6929, Train Acc: 78.84%, Val Loss: 0.6720, Val Acc: 79.92%
Epoch 9/50, Train Loss: 0.6546, Train Acc: 80.28%, Val Loss: 0.6340, Val Acc: 81.16%
Epoch 10/50, Train Loss: 0.6252, Train Acc: 81.56%, Val Loss: 0.6058, Val Acc: 82.36%
Epoch 11/50, Train Loss: 0.6029, Train Acc: 81.92%, Val Loss: 0.5785, Val Acc: 82.91%
Epoch 12/50, Train Loss: 0.5777, Train Acc: 83.83%, Val Loss: 0

In [5]:
# Save the model
torch.save(model.state_dict(), f'models/gta_resnet50_lr{learning_rate}_{epochs}_epochs.pth')

In [6]:
#Clear Gpu mem.
del optimizer
del model
torch.cuda.empty_cache()